## Python Memory MAnagement

It involves a combination of automatic garbage collenction, reference counting, and variious internal optimizations to efficient manage memory allocation and deallocation. Understanding these mechanism can help developers write more efficient and robust apploications.

1. Key Concept in Python Memory Management
2. Memory Alllocation and Deallocation
3. reference counting
4. Garbage dcollection
5. The gc Module

### Reference count —

a **counter** of how many references point to that object. When an object’s reference count drops to zero, it means no one is using it anymore, and it can be safely deleted from memory.

In [1]:
import sys

a=[]

print(sys.getrefcount([]))  ## output: 1 (one because of getrefcount() function)

print(sys.getrefcount(a))  ## output: 2 (one for variable a and second for getrefcount() function)

1
2


In [2]:
b=a

print(sys.getrefcount(b)) ## output: 3

3


In [3]:
## delete reference b : memory will dealoocated
del b

In [4]:
print(sys.getrefcount(b))  ## NameError: name 'b' is not defined

NameError: name 'b' is not defined

In [5]:
print(sys.getrefcount(a))

2


## Garbage Collection (gc)
python includes a cyclic garbage collector to habdle reference cycle.<br>
Reference cycle occures when object referencne each other, preventing their reference counts from from reaching Zero.

In [2]:
import gc

## enabling garbage collection
gc.enable()

In [7]:
## disable garbage collection
gc.disable()

## collect():

collect() is used to manually trigger garbage collection in Python. It returns the number of unreachable objects that were found and successfully collected (i.e., cleaned up from memory).

## 📌 What It Represents
When you call gc.collect(), here’s what happens under the hood:

1. Python scans all generations (0, 1, and 2).
2. It tries to detect cycles of objects that are no longer reachable.
3. It breaks those cycles and frees the memory.
4. It returns the number of objects it was able to collect.

#### Python GC ki 3 Generations kya hain?
* generation 0
* generation 1
* generation 2


#### Kaise Kaam Karta Hai?
* Jab aap koi object create karte ho, vo generation 0 me jata hai.
* Agar vo object generation 0 se bach jata hai (yaani GC ne usko nahi delete kiya), to vo generation 1 me chala jata hai.
* Fir agar vo waha se bhi bacha raha, to vo generation 2 me chala jata hai.

In [8]:
## get unreachable objects list
print(gc.garbage)

[]


In [9]:
## garbage collector : The number of unreachable objects is returned.
import gc

unreachable = gc.collect()
print(f"Collected {unreachable} unreachable objects.")

Collected 522 unreachable objects.


In [10]:
import gc
## The number of unreachable objects is returned.

print(gc.get_count())  ## Output: (gen0_count, gen1_count, gen2_count)
# e.g., (450, 12, 2)

(102, 0, 0)


In [11]:
### specify a generation (0, 1, or 2):

print(gc.collect(0))  # Only collect Gen 0
print(gc.collect(1))  # Collect Gen 0 and Gen 1
print(gc.collect(2))  # Collect all generations (default)

0
0
0


## Memory Management Best Practices

#### 1. Use Local Varible:
* local varibles have a shorter lifespan and are freed sooner than global varibles.
#### 2. Avoid Circular Referances:
* Circular references can lead memory leak if not properly managed.
#### 3. Explicitly Delete Objects:
* Use the *del* keyword to delete variables and objectes explicitly.
#### 4. Use Generator:
* Generator produce items one at a time and only keep one item in memory at a time, making them memory effcient.
#### 4. Profile Memory Usages:
* Use memory Profiling tools like tracemalloc and memory_profiler to identify memory leaks and optimize memory usage.

In [94]:
## Handling circular reference

class MyClass:
    def __init__(self, name):
        self.name =  name
        print(f" obj {self.name} is created")
    
    def __del__(self):
        print(f" object {self.name} is deleted")



obj1 = MyClass("obj1")
obj2 = MyClass("obj2")

## create circular references
obj1.ref = obj2
obj2.ref = obj1
## Now, obj1 holds a reference to obj2, and obj2 holds a reference to obj1. 
## So even if you delete obj1 and obj2 (like below), these objects still reference each other,
## meaning their reference count doesn’t drop to zero.

del obj1
del obj2
## This removes the names obj1 and obj2 from the current scope.
## However, the objects are not immediately destroyed because they are still referring to each other via .ref.
## So, their __del__ methods are not called at this point.

gc.collect()
## This manually triggers Python's garbage collector, which is smart enough to detect circular references.
## It sees that the objects can’t be accessed anymore (no external references),
## so it cleans them up and calls their destructors (__del__).

 obj obj1 is created
 obj obj2 is created
 object obj1 is deleted
 object obj2 is deleted


9

In [116]:
## Generators For Memory Efficiency

def generate_number(n):
    for i in range(n):
        yield i
    

## Using the generator
for num in generate_number(10):
    print(num, end=" ")

0 1 2 3 4 5 6 7 8 9 